In [1]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# データの読み込みと前処理
def load_data(directory):
    texts, labels = [], []
    for label, category in enumerate(["dokujo-tsushin", "it-life-hack"]):
        category_dir = os.path.join(directory, category)
        for file in os.listdir(category_dir):
            if os.path.isfile(os.path.join(category_dir, file)):
                with open(os.path.join(category_dir, file), 'r', encoding='utf-8') as f:
                    lines = f.readlines()[2:]  # 最初の2行をスキップ
                    text = ''.join(lines).strip()
                    texts.append(text)
                    labels.append(label)
    return texts, labels

home_dir = os.path.expanduser("~")
data_dir = os.path.join(home_dir, "data/livedoor-text")
texts, labels = load_data(data_dir)

# データセットの分割
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.4, random_state=1)
val_texts, test_texts, val_labels, test_labels = train_test_split(test_texts, test_labels, test_size=0.5, random_state=1)

# データセットの準備
class LivedoorDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

train_dataset = LivedoorDataset(train_encodings, train_labels)
val_dataset = LivedoorDataset(val_encodings, val_labels)
test_dataset = LivedoorDataset(test_encodings, test_labels)

print(f"{len(train_dataset)=}")
print(f"{train_texts[0]=}")
print(f"{train_labels[0]=}")


len(train_dataset)=1045
train_texts[0]='結婚しても働くのはなぜ？ 既婚女性のつぶやき\n\u3000「彼の収入が少ないから私も働かなければならないし、それを思うと結婚はもう少し先でもいいかな」と結婚を躊躇する独女がいる。彼女は彼の収入だけで暮らせるのなら、仕事は今すぐにでも辞めたいらしい。つまり専業主婦志望なのだが、彼の年収を聞いて首を傾げた。\n\n\u3000この金額で本当に生活ができないのだろうか？ \n\n\u3000かつて専業主婦が多かった時代、主婦の働き先はなく、今月もかつかつだとこぼしながらも、夫の稼ぎだけで暮らしていた家庭が多かった。しかし今は不況で夫の収入が減ったとはいえ、外食、ブランド品購入、安いツアーとはいえ海外旅行にも行っている。食べるだけで精一杯の昔に比べれば、ものすごく贅沢ではないだろうか？\n\u3000\n\u3000成人した二人の子供がいる専業主婦の紀世子さん（56 歳）は、「今は専業主婦がセレブのように言われますけど、私はブランド品も持ったことがなければ、家族で海外旅行にも行ったことがないんですよ。夫の収入だけで充分とはいいませんけど、贅沢さえしなければ毎月何とかなったものです」という。\n\u3000\n\u3000子供が小学校に入学すると、塾の費用を捻出するためにパートに行く主婦もいたが、紀世子さんの家庭はご主人の方針で塾には一切通わせず、兄は水泳、妹は習字と、週に一度の習い事に通わせただけだそうだ。\n\n\u3000「私立中学受験で塾に通わせているご家庭は大変そうでしたよ。塾の費用が一か月5万円と聞いてびっくりしました。そこまでして私立に行かせて、その後も莫大な教育費がかかるのに大変だとあと思いました」\n\n\u3000紀世子さんの長女は私立の女子大学に入学したが、中学・高校から持ち上がりできた友人には小学校の時の同級生もいる。「中高一貫教育の必要性はよく分かりませんが、結局同じ大学に通うなら何も高い教育費を払って中学から行く必要がないのでは？」これは私の考えですがと紀世子さん。\n\n\u3000仕事に生きがいを持ち自分のために働いている主婦もいるが、家族で海外旅行に行ったり外食をしたり、生活水準を上げるために働いている主婦もいる。自分の稼ぎでブランド品を買う主婦もいるが、やはり

In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertJapaneseTokenizer
import numpy as np

# 教師モデルとトークナイザの読み込み
model_path = "./finetuned_bert_japanese"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v3")

# 教師モデルのembedding layerのパラメータを取得
vocab_size = model.bert.embeddings.word_embeddings.num_embeddings
embedding_dim = model.bert.embeddings.word_embeddings.embedding_dim
print(f"Vocabulary size: {vocab_size}, Embedding dimension: {embedding_dim}")

# データセットの準備（前のステップで作成したものを再利用）
# ...

# データセットから教師モデルの出力を取得する関数
def get_model_outputs(dataset):
    dataloader = DataLoader(dataset, batch_size=8)
    model.eval()
    outputs = []
    for batch in dataloader:
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = torch.nn.functional.softmax(logits, dim=-1)
            outputs.append(probs.cpu().numpy())
    return np.concatenate(outputs, axis=0)

# train, val, testデータセットから教師モデルの出力を取得
train_outputs = get_model_outputs(train_dataset)
val_outputs = get_model_outputs(val_dataset)
test_outputs = get_model_outputs(test_dataset)

# 教師モデルの出力を保存（必要に応じて）
np.save('train_outputs.npy', train_outputs)
np.save('val_outputs.npy', val_outputs)
np.save('test_outputs.npy', test_outputs)

# 教師モデルをメモリから削除
del model


Vocabulary size: 32768, Embedding dimension: 768
